In [1]:
%reset -fs


# Gaussian Mixture Model


Plot the confidence ellipsoids of a mixture of two Gaussians
obtained with Expectation Maximisation (``GaussianMixture`` class) and
Variational Inference (``BayesianGaussianMixture`` class models with
a Dirichlet process prior).

Both models have access to five components with which to fit the data. Note
that the Expectation Maximisation model will necessarily use all five
components while the Variational Inference model will effectively only use as
many as are needed for a good fit. Here we can see that the Expectation
Maximisation model splits some components arbitrarily, because it is trying to
fit too many components, while the Dirichlet Process model adapts it number of
state automatically.

Another advantage of the Dirichlet process model is that it can fit
full covariance matrices effectively even when there are less examples
per cluster than there are dimensions in the data, due to
regularization properties of the inference algorithm.




In [2]:
%matplotlib
import itertools
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats
from sklearn import mixture

import wget
import pandas as pd
import time
color_iter = itertools.cycle(['navy', 'c', 'cornflowerblue', 'gold',
                              'darkorange'])


def plot_results(X, Y_, means, covariances, index, title):
    splot = plt.subplot(2, 1, 1 + index)
    for i, (mean, covar, color) in enumerate(zip(
            means, covariances, color_iter)):
        v, w = linalg.eigh(covar)
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        u = w[0] / linalg.norm(w[0])
        # as the DP will not use every component it has access to
        # unless it needs it, we shouldn't plot the redundant
        # components.
        if not np.any(Y_ == i):
            continue
        plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], .8, color=color)

        # Plot an ellipse to show the Gaussian component
        angle = np.arctan(u[1] / u[0])
        angle = 180. * angle / np.pi  # convert to degrees
        ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle, color=color)
        ell.set_clip_box(splot.bbox)
        ell.set_alpha(0.5)
        splot.add_artist(ell)

    plt.xlim(-9., 5.)
    plt.ylim(-3., 6.)
    plt.xticks(())
    plt.yticks(())
    plt.title(title)

def gauss_mix(X,n,mode,plot=False):
    tic = time.time()
    if mode =='dpgmm':
        # Fit a Dirichlet process Gaussian mixture using five components
        dpgmm = mixture.BayesianGaussianMixture(n_components=n,
                                                covariance_type='full',random_state=1,max_iter=5000).fit(X)
        if plot == True:
            plot_results(X, dpgmm.predict(X), dpgmm.means_, dpgmm.covariances_, 1,
                    'Bayesian Gaussian Mixture with a Dirichlet process prior')

        print('Bayesian Gaussian Mixture with a Dirichlet process prior')
        centers = np.empty(shape=(dpgmm.n_components, X.shape[1]))
        kp = np.zeros(1)
        for i in range(dpgmm.n_components):
            density = scipy.stats.multivariate_normal(cov=dpgmm.covariances_[i], mean=dpgmm.means_[i]).logpdf(X)
            centers[i, :] = X[np.argmax(density)]
            kp = np.vstack((kp,i))
        if plot == True:
            plt.scatter(centers[:, 0], centers[:, 1], s=20,marker='*')
            plt.show()
        print('Full covariance matrices')
        print(dpgmm.covariances_)
        p=dpgmm.predict_proba(X)
        q=dpgmm.predict(X)
    else:
        # Fit a Gaussian mixture with EM using five components
        gmm = mixture.GaussianMixture(n_components=n, covariance_type='full',random_state=1).fit(X)
        if plot == True:
            plot_results(X, gmm.predict(X), gmm.means_, gmm.covariances_, 0,
                   'Gaussian Mixture')

        print('Gaussian Mixture Results')
        centers = np.empty(shape=(gmm.n_components, X.shape[1]))
        kp = np.zeros(1)
        for i in range(gmm.n_components):
            density = scipy.stats.multivariate_normal(cov=gmm.covariances_[i], mean=gmm.means_[i]).logpdf(X)
            centers[i, :] = X[np.argmax(density)]
            kp = np.vstack((kp,i))
        if plot == True:
            plt.scatter(centers[:, 0], centers[:, 1], s=20,marker='*',color='red')
            plt.show()

        print('Full covariance matrices')
        print(gmm.covariances_)
        p=gmm.predict_proba(X)
        q=gmm.predict(X)

    if np.shape(X)[1] == 2:
        kp = np.column_stack((kp[1:],centers[:, 0]))
        kp = np.column_stack((kp, centers[:, 1]))
    elif np.shape(X)[1] == 3:
        kp = np.column_stack((kp[1:],centers[:, 0]))
        kp = np.column_stack((kp, centers[:, 1]))
        kp = np.column_stack((kp, centers[:, 2]))
    elif np.shape(X)[1] == 4:
        kp = np.column_stack((kp[1:],centers[:, 0]))
        kp = np.column_stack((kp, centers[:, 1]))
        kp = np.column_stack((kp, centers[:, 2]))
        kp = np.column_stack((kp, centers[:, 3]))
    elif np.shape(X)[1] == 5:
        kp = np.column_stack((kp[1:],centers[:, 0]))
        kp = np.column_stack((kp, centers[:, 1]))
        kp = np.column_stack((kp, centers[:, 2]))
        kp = np.column_stack((kp, centers[:, 3]))
        kp = np.column_stack((kp, centers[:, 4]))
        
    print('Centeroids')
    print(kp)
    toc = time.time()
    print('GMM Run time : ',toc-tic,' seconds')
    return p,q



Using matplotlib backend: TkAgg


# GMM on GAIA EDR3 FOV
### Data Prepration

In [ ]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from zero_point import zpt

df1 = pd.read_csv('FOV_6791.csv')

df1 = df1[df1.astrometric_params_solved>3] # removing 3 parameter solutions
#zeropoint from lindgren 2020, adsurl = https://ui.adsabs.harvard.edu/abs/2020arXiv201201742L
zpt.load_tables()
df1['zero_point'] = df1.apply(zpt.zpt_wrapper,axis=1)
df1.parallax = df1.parallax - df1.zero_point # zeropoint correction

print('original data: ',len(df1))
df1 = df1[np.isfinite(df1['pmra'])]
df1 = df1[np.isfinite(df1['pmdec'])]
print('after proper-motion nan correction: ',len(df1))
df1 = df1[np.isfinite(df1['parallax'])]
print('after parallax nan correction:', len(df1))
df1 = df1[np.isfinite(df1['radial_velocity'])]
print('after radial velocity nan correction:', len(df1))

#uncertainity filter
df1['un_plx'] = abs(df1['parallax_error']/df1['parallax'])*100
df1 = df1[df1.un_plx<=50]
print('after parallax uncertainity filter: ',len(df1))
df1['un_pmra'] = abs(df1['pmra_error']/df1['pmra'])*100
df1 = df1[df1.un_pmra<=50]
print('after pmra uncertainity filter: ',len(df1))
df1['un_pmdec'] = abs(df1['pmdec_error']/df1['pmdec'])*100
df1 = df1[df1.un_pmdec<=50]
print('after pmdec uncertainity filter: ',len(df1))

#filtering forground and background
df1 = df1[(df1.parallax>0)&(df1.parallax<1)]
print('after parallax field filter: ',len(df1))


## GMM calculations and probability estimation 

In [ ]:
#df1 = df1[pd.notnull((df1.parallax))&(pd.notnull(df1.pmra))&pd.notnull((df1.pmdec))]

p1 = df1.pmra
p2 = df1.pmdec
p3 = df1.parallax
p4 = df1.radial_velocity
p5 = df1.ra
p6 = df1.dec

X = np.column_stack([p1,p2])
X = np.column_stack([X,p3])
X = np.column_stack([X,p4])
X = np.column_stack([X,p5])
X = np.column_stack([X,p6])


XX = np.column_stack([df1.designation,X])
print('Final data length before gmm : ', len(df1))
#GMM
p,q = gauss_mix(X,15,'dpgmm')
print(p)
print(q)
#choose the right centroid from priors
c = input('Enter the centroid: ')

#Stacking Data
gdata = np.column_stack([XX,q])
pb = np.zeros(len(p))
avgp1 = np.zeros(len(p))
avgp2 = np.zeros(len(p))
A = 'ID,GM,P,gmm_prob'
gmm_prob = np.zeros(1)
for j in range(len(p)):
    pb[j] = p[j][q[j]]
    #A0 = str(XX[:,0][j])+','+str(q[j])+','+str(p[j][q[j]])+','+str(p[j][int(c)])
    #A = np.append(A,A0)
    gmm_prob = np.vstack((gmm_prob,p[j][int(c)]))
gdata = np.column_stack([gdata,pb])
df1['gmm_prob'] = gmm_prob[1:]
df1.to_csv('6791_GEDR3_GMM_5Dnf.csv')
#np.savetxt('6791_gmm_prob',A,fmt='%s')
print(len(p))

## MATCHING the V*6791 catalogue

In [ ]:
%reset -fs
import pandas as pd
import numpy as np
df = pd.read_csv('6791_GEDR3_GMM_5Dnf.csv')
dq = pd.read_csv('NGC6791.csv')
match = str('ID')+','+str('GAIA_ID')+','+str('prob')
for i in range(len(dq)):
    df1 = df[df.designation==dq.GAIA_ID[i]]
    if len(df1)>=1:
        df1 = df1.head(1)
        match0 = str(dq.ID[i])+','+str(dq.GAIA_ID[i])+','+str(df1.gmm_prob.item())
        match = np.vstack((match,match0))
    else:
        match0 = str(dq.ID[i])+','+str(dq.GAIA_ID[i])+','
        match = np.vstack((match,match0))
np.savetxt("V6791_GEDR3_GMM_5Dnf.csv",match,fmt="%s")

## Determining general parameters from members

In [ ]:
#%reset -fs
import pandas as pd
import numpy as np
#PROBABILITY DECISION
df = pd.read_csv('6791_GEDR3_GMM_5Dnf.csv')


pt = 0.9
print('Almost certain members : ', len(df[df.gmm_prob>pt]))

df = df[df.gmm_prob > pt]

p1 = df.pmra
p2 = df.pmdec
p3 = df.parallax
p4 = df.ra
p5 = df.dec

X = np.column_stack([p1,p2])
X = np.column_stack([X,p3])
X = np.column_stack([X,p4])
X = np.column_stack([X,p5])

#GMM
p,q = gauss_mix(X,10,'dpgmm')
print(p)
print(q)
#choose the right centroid from priors
c = input('Enter the centroid: ')

print('ra: ',np.mean(p5),np.std(p5)/2036., ', dec: ',np.mean(p4),np.std(p4)/2036, ', pmra: ',np.mean(p1),np.std(p1)/2036, ', pmdec: ',np.mean(p2),np.std(p2)/2036, ', parallax:',np.mean(p3),np.std(p3
)/2036)